In [5]:
import math
import torch
import numpy as np
import gpytorch
import pandas as pd
import seaborn as sns
import os
import matplotlib 
matplotlib.use('Agg')
from matplotlib import pyplot as plt
import sklearn
from sklearn.model_selection import KFold

%matplotlib inline
%load_ext autoreload
%autoreload 2

import imblearn
print("imblearn version: ",imblearn.__version__)
from imblearn.over_sampling import SMOTE

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


from sklearn.metrics import confusion_matrix
import itertools


from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score

from sklearn.ensemble import RandomForestClassifier
from rdkit import Chem
import sys
sys.path.append('../')
import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
imblearn version:  0.12.0


In [2]:
# from sklearn 0.19.2 documentation:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.colorbar(shrink=0.7)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [7]:
split_path = '../../../data/NEK_data_4Berkeley/NEK2/'
file = 'NEK2_1_uM_min_50_pct_binding_5fold_random_imbalanced.csv'
all_fold_df = pd.read_csv(split_path+file)

In [8]:

smile_prop_train = all_fold_df[all_fold_df['fold'] != 'fold5'].drop(columns=['fold'])
smile_prop_test = all_fold_df[all_fold_df['fold'] == 'fold5'].drop(columns=['fold'])

mol_train = []
for i in smile_prop_train.index:
    m = Chem.MolFromSmiles(smile_prop_train.loc[i, 'base_rdkit_smiles'])
    if m:
        m.SetProp("_Name", smile_prop_train.loc[i, 'compound_id'])
        m.SetProp("Active", str(smile_prop_train.loc[i, 'active']))
        mol_train.append(utils.normalize(m))

mol_test = []
for i in smile_prop_test.index:
    m = Chem.MolFromSmiles(smile_prop_test.loc[i, 'base_rdkit_smiles'])
    if m:
        m.SetProp("_Name", smile_prop_test.loc[i, 'compound_id'])
        m.SetProp("Active", str(smile_prop_test.loc[i, 'active']))
        mol_test.append(utils.normalize(m))


[14:37:21] Initializing MetalDisconnector
[14:37:21] Running MetalDisconnector
[14:37:21] Initializing Normalizer
[14:37:21] Running Normalizer
[14:37:21] Running LargestFragmentChooser
[14:37:21] Running Uncharger
[14:37:21] Initializing MetalDisconnector
[14:37:21] Running MetalDisconnector
[14:37:21] Initializing Normalizer
[14:37:21] Running Normalizer
[14:37:21] Initializing MetalDisconnector
[14:37:21] Running MetalDisconnector
[14:37:21] Initializing Normalizer
[14:37:21] Running Normalizer
[14:37:21] Running LargestFragmentChooser
[14:37:21] Running Uncharger
[14:37:21] Initializing MetalDisconnector
[14:37:21] Running MetalDisconnector
[14:37:21] Initializing Normalizer
[14:37:21] Running Normalizer
[14:37:21] Initializing MetalDisconnector
[14:37:21] Running MetalDisconnector
[14:37:21] Initializing Normalizer
[14:37:21] Running Normalizer
[14:37:21] Running LargestFragmentChooser
[14:37:21] Running Uncharger
[14:37:21] Initializing MetalDisconnector
[14:37:21] Running MetalD